# Neural Net vs. Goldman
Can an LTSM Neural Net trained on fundamentals extracted from edgar XBRL from the S&P500 pick the same long and short list as Goldman's hedgefund meta list (appologies to my high school english teacher for the run on sentence)

## Data Sources
Download price and fundamental data for the S&P 500 using [pystock-crawler](https://github.com/eliangcs/pystock-crawler):  

`pystock-crawler reports ../tickers.csv -o ../reports.csv ../reports.log`  
`pystock-crawler prices ../tickers.csv -o ../prices.csv -l ../prices.log`

REMIND: Use pystock-crawler symbols to get symbols for training input...

X = 4 quarters of fundamental data and whether stock was up or down from the prior quarter y = whether the stock was up or down x days after the period that X was comprised of

In [184]:
import pandas
prices = pandas.read_csv('prices.csv', parse_dates=['date'], index_col=1)
reports = pandas.read_csv('reports.csv', parse_dates=['date'], index_col=1)
symbols = pandas.read_csv('symbols.csv')

In [191]:
s = "MMM"
r = reports[reports.symbol == s].sort(ascending=True)
p = prices[prices.symbol == s].sort(ascending=True)

In [211]:
p.ix['2015-05-12'].close

160.07001

In [233]:
# Add closing price and delta from prior period report onto each
X = pandas.DataFrame()
for s in symbols['symbol'][0:4]:
    try:
        r = reports[reports.symbol == s].sort(ascending=True)
        p = prices[prices.symbol == s].sort(ascending=True)
        r['close'] = r.index.map(lambda x: p.ix[p.index[p.index.searchsorted(x)]]['close'])
        r['delta'] = (r['close']-r['close'].shift()).fillna(0)
        X = X.append(r)
    except:
        print "Problems parsing", s
print "Found", len(X['symbol'].unique()), "symbols"

Found 4 symbols


In [ ]:
# Split into train/test sets
# Should use sklearn.cross_validation.StratifiedShuffleSplit to try and maintain industry sector % in each
# Or bin by financial size http://www.gregreda.com/2013/10/26/using-pandas-on-the-movielens-dataset/